In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/kyou/Library/CloudStorage/Dropbox-Personal/kisungyou.github.io':
  os.chdir(r'/Users/kyou/Library/CloudStorage/Dropbox-Personal/kisungyou.github.io')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"/Users/kyou/miniconda3/lib/python3.11/importlib/_bootstrap.py": 1688564785.7364845, "/Users/kyou/miniconda3/lib/python3.11/importlib/_bootstrap_external.py": 1688564785.7473316, "/Users/kyou/miniconda3/lib/python3.11/zipimport.py": 1688564782.2649462, "/Users/kyou/miniconda3/lib/python3.11/codecs.py": 1688564781.0550086, "/Users/kyou/miniconda3/lib/python3.11/encodings/aliases.py": 1688564783.6747382, "/Users/kyou/miniconda3/lib/python3.11/encodings/__init__.py": 1688564783.6672077, "/Users/kyou/miniconda3/lib/python3.11/encodings/utf_8.py": 1688564784.717895, "/Users/kyou/miniconda3/lib/python3.11/abc.py": 1688564780.8473778, "/Users/kyou/miniconda3/lib/python3.11/io.py": 1688564781.4512396, "/Users/kyou/miniconda3/lib/python3.11/stat.py": 1688564781.9388707, "/Users/kyou/miniconda3/lib/python3.11/_collections_abc.py": 1688564780.7103467, "/Users/kyou/miniconda3/lib/python3.11/genericpath.py": 1688564781.304632, "/Users/kyou/miniconda3/lib/python3.11/posixpath.py": 1688564781.644509

In [2]:
import yaml
from IPython.display import display, Markdown, HTML

# load centralized author URLs
with open('list_authors.yaml') as file:
    author_urls = yaml.safe_load(file)
    
centralized_author_urls = author_urls.get("author_urls", {})

def readable_list(_s):
  authors_with_links = [
    f"<a href='{centralized_author_urls.get(aut, '#')}' target='_blank'>{aut}</a>" if aut in centralized_author_urls else aut 
    for aut in _s
  ]
  if len(authors_with_links) < 3:
    return ' and '.join(map(str, authors_with_links))
  *a, b = authors_with_links
  return f"{', '.join(map(str, a))}, and {b}"
  # if len(_s) < 3:
  #   return ' and '.join(map(str, _s))
  # *a, b = _s
  # return f"{', '.join(map(str, a))}, and {b}"

def button(url, str, icon):
    icon_base = icon[:2]
    return f"""<a class="btn btn-outline-dark btn-sm", href="{url}" target="_blank" rel="noopener noreferrer">
        <i class="{icon_base} {icon}" role='img' aria-label='{str}'></i>
        {str}
    </a>"""

yaml_data = yaml.safe_load(open("list_publication.yaml"))
pub_strs = {"pubs": {}, "wps": {}}
for _, data in yaml_data.items():
    title_str = data["title"]
    #print(f"Processing: {title_str}")  # Debugging step
    
    authors = data.get("authors", ["me"])
    authors = [aut if aut != "me" else "<strong>KY</strong>" for aut in authors]
    author_str = readable_list(authors)
    year_str = data["year"]

    buttons = []
    preprint = data.get("preprint")
    if preprint is not None:
        buttons.append(button(preprint, "Preprint", "bi-file-earmark-pdf"))

    github = data.get("github")
    if github is not None:
        buttons.append(button(github, "Code", "bi-github"))

    pub_url = data.get("published_url")
    venue = data.get("venue")
    flag  = data.get("flag")
    working_paper = pub_url is None
    
    pub_str = f'{author_str}. ({year_str}) "{title_str}."'

    # DIFFERENT TYPES
    # 1. published (anywhere)
    # 2. preprint  

    # TYPES-strings
    if flag=="published":
      pub_str = f"<b>{title_str}</b><br> {author_str}. <br> <em>{venue}</em>."
    elif flag=="preprint":
      pub_str = f"<b>{title_str}</b><br> {author_str} ({year_str})."
    
    if flag == "published":
      if pub_url:
        buttons.append(button(pub_url, "Publisher's site", "ai-archive"))
      # Ensure you append to the list even if the year already exists
      if year_str not in pub_strs["pubs"]:
        pub_strs["pubs"][year_str] = []
      pub_strs["pubs"][year_str].append(
        "<li class='list-group-item'>" + pub_str + "<br>" + " ".join(buttons) + "</li>"
      )
    # For preprints
    elif flag == "preprint":
      if year_str not in pub_strs["wps"]:
        pub_strs["wps"][year_str] = []
      pub_strs["wps"][year_str].append(
        "<li class='list-group-item'>" + pub_str + "<br>" + " ".join(buttons) + "</li>"
      )

In [3]:
#| label: journal-year
#| output: asis
for year in sorted(pub_strs["pubs"].keys(), reverse=True):
  display(Markdown(f"### {year}"))
  display(HTML(
    "<ul class='list-group list-group-flush'>"
    + "\n".join(pub_strs["pubs"][year])
    + "</ul>"
  ))

### 2024

### 2023

### 2022

### 2021

### 2020

### 2019

### 2016

In [4]:
#| label: preprint-year
#| output: asis
for year in sorted(pub_strs["wps"].keys(), reverse=True):
  #display(Markdown(f"### {year}" + "{#" + f"not-published-{year}" + "}"))
  display(HTML(
    "<ul class='list-group list-group-flush'>" + '\n'.join(pub_strs["wps"][year]) + "</ul>"
  ))